In [6]:
#pip install langchain-community pypdf

In [7]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [8]:
print(documents)

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'), Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]


In [9]:
for doc in documents:
    print(doc.page_content)


Dogs are great companions, known for their loyalty and friendliness.
Cats are independent pets that often enjoy their own space.


In [10]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "Chapter 01.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

19


In [11]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

IN2311:Operating systems
Roshani Wijesuriya

{'producer': 'Microsoft® PowerPoint® LTSC', 'creator': 'Microsoft® PowerPoint® LTSC', 'creationdate': '2024-10-12T10:35:42+05:30', 'title': '', 'author': 'Roshani Wijesuriya', 'moddate': '2024-10-12T10:35:42+05:30', 'source': 'Chapter 01.pdf', 'total_pages': 19, 'page': 0, 'page_label': '1'}


In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

20

In [13]:
#pip install -qU langchain-google-genai

In [14]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [15]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 3072

[0.01898874156177044, -0.008987678214907646, 0.009013982489705086, -0.03462599590420723, -0.01431077066808939, 0.009711828082799911, -0.006494375877082348, -0.011864026077091694, -0.008294306695461273, 0.010139936581254005]
